# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the XES file
    dataframe_log = pm4py.read_xes('../../data/logs/InternationalDeclarations.xes')

    # If 'log' is already a DataFrame, add the @@case_index column directly
    case_indices = {case_id: idx for idx, case_id in enumerate(dataframe_log['case:concept:name'].unique())}
    dataframe_log['@@case_index'] = dataframe_log['case:concept:name'].map(case_indices)
    
     # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

/opt/anaconda3/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/6449 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,case:Permit TaskNumber,...,case:Permit OrganizationalEntity,case:travel permit number,case:Permit RequestedBudget,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,@@case_index
0,rv_travel permit 76455_6,STAFF MEMBER,Start trip,2016-10-04 22:00:00+00:00,EMPLOYEE,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
1,rv_travel permit 76455_7,STAFF MEMBER,End trip,2016-10-04 22:00:00+00:00,EMPLOYEE,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
2,st_step 76459_0,STAFF MEMBER,Permit SUBMITTED by EMPLOYEE,2017-04-06 11:32:10+00:00,EMPLOYEE,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
3,st_step 76460_0,STAFF MEMBER,Permit FINAL_APPROVED by SUPERVISOR,2017-04-06 11:32:28+00:00,SUPERVISOR,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
4,st_step 76461_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-04-07 11:38:14+00:00,EMPLOYEE,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72146,st_step 13239_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-18 14:06:50+00:00,EMPLOYEE,travel permit number 13227,UNKNOWN,0.000000,0.000000,task 427,...,organizational unit 65455,travel permit number 13227,1727.559756,declaration 13232,travel permit 13226,travel permit 13226,budget 147449,UNKNOWN,0.000000,6448
72147,st_step 13241_0,STAFF MEMBER,Declaration REJECTED by ADMINISTRATION,2018-12-18 14:06:57+00:00,ADMINISTRATION,travel permit number 13227,UNKNOWN,0.000000,0.000000,task 427,...,organizational unit 65455,travel permit number 13227,1727.559756,declaration 13232,travel permit 13226,travel permit 13226,budget 147449,UNKNOWN,0.000000,6448
72148,st_step 13240_0,STAFF MEMBER,Declaration REJECTED by EMPLOYEE,2018-12-19 13:05:36+00:00,EMPLOYEE,travel permit number 13227,UNKNOWN,0.000000,0.000000,task 427,...,organizational unit 65455,travel permit number 13227,1727.559756,declaration 13232,travel permit 13226,travel permit 13226,budget 147449,UNKNOWN,0.000000,6448
72149,rv_travel permit 13226_6,STAFF MEMBER,Start trip,2019-02-18 23:00:00+00:00,EMPLOYEE,travel permit number 13227,UNKNOWN,0.000000,0.000000,task 427,...,organizational unit 65455,travel permit number 13227,1727.559756,declaration 13232,travel permit 13226,travel permit 13226,budget 147449,UNKNOWN,0.000000,6448


# Insert Start & End markers

In [2]:
# Define a function to insert start and end markers
def add_markers(df):
    # Identify unique case indices
    case_indices = df['@@case_index'].unique()
    
    # Prepare a container for new DataFrame rows
    new_rows = []
    
    # Iterate over each case index to add start and end markers
    for case_index in case_indices:
        # Create a start marker row with all columns except @@case_index set to 'Start'
        start_row = {col: 'Start' if col != '@@case_index' else case_index for col in df.columns}
        
        # Create an end marker row with all columns except @@case_index set to 'End'
        end_row = {col: 'End' if col != '@@case_index' else case_index for col in df.columns}
        
        # Append start row, rows for the current case, and end row
        new_rows.append(start_row)
        new_rows.extend(df[df['@@case_index'] == case_index].to_dict('records'))
        new_rows.append(end_row)
    
    # Convert the list of rows into a DataFrame
    return pd.DataFrame(new_rows)

# Apply the function to add start and end markers to the dataframe
modified_dataframe = add_markers(dataframe_log)

# Preprocess

In [3]:
codes, uniques = pd.factorize(modified_dataframe['concept:name'])
modified_dataframe['concept:name'] = codes

### Padding for Cases with less then 5 events

In [4]:
# Calculating the frequency of each unique value in '@@case_index'
frequency = modified_dataframe['@@case_index'].value_counts()

# Finding the minimum occurrence
min_occurrence = frequency.min()

min_occurrence

5

# Generate sliding windows

In [5]:
df_activity = modified_dataframe[['concept:name', '@@case_index']]

In [6]:
def generate_sliding_windows(df, case_id_column='@@case_index', window_size=5):
    windows = []
    targets = []
    case_indices = []

    # Iterate over each unique case
    for case_id in df[case_id_column].unique():
        # Extract the case
        case_data = df[df[case_id_column] == case_id]
        
        # Convert case_data to a NumPy array and drop the case_id_column
        case_data_array = case_data.drop(columns=[case_id_column]).to_numpy()

        # Adjusting the condition to correctly reflect window_size without needing an additional +1
        # Now it correctly considers window_size as including the target event
        if len(case_data_array) >= window_size:
            # Adjust the loop to generate sliding windows of size window_size - 1 for the inputs and use the next event as the target
            for i in range(len(case_data_array) - window_size + 1):
                # window now has window_size - 1 events
                window = case_data_array[i:i + window_size - 1]
                # The target is the event immediately following the window
                target = case_data_array[i + window_size - 1]
                windows.append(window)
                targets.append(target)
                case_indices.append(case_id)  # Store the case_id corresponding to the window

    # Convert lists to numpy arrays for easier handling and to ensure they are two-dimensional
    windows_array = np.array(windows)
    targets_array = np.array(targets)
    case_indices_array = np.array(case_indices)
    
    return windows_array, targets_array, case_indices_array

In [7]:
windows_activity, targets_activity, case_indices = generate_sliding_windows(df_activity)

# LSTM

### Architecture

- id not included because LSTm could not learn

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Assuming these values as placeholders, replace them with actual counts from your data
num_activities = modified_dataframe['concept:name'].nunique()

embedding_dim_activity = 50
time_steps = 4

# Input layer for activities
input_activity = Input(shape=(time_steps,), name='input_activity')

# Embedding layer for activities
embedding_activity = Embedding(input_dim=num_activities, output_dim=embedding_dim_activity, input_length=time_steps)(input_activity)

# LSTM layer for activities
lstm_activity = LSTM(25, return_sequences=False)(embedding_activity)

# Output layer for predicting the next activity
output_activity = Dense(num_activities, activation='softmax', name='output_activity')(lstm_activity)

# Create and compile the model
model = Model(inputs=input_activity, outputs=output_activity)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

2024-08-19 21:48:28.781288: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_activity (InputLayer  [(None, 4)]               0         
 )                                                               
                                                                 
 embedding (Embedding)       (None, 4, 50)             1800      
                                                                 
 lstm (LSTM)                 (None, 25)                7600      
                                                                 
 output_activity (Dense)     (None, 36)                936       
                                                                 
Total params: 10336 (40.38 KB)
Trainable params: 10336 (40.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Data Splitting

In [9]:
from sklearn.model_selection import train_test_split

# Split the activity data
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

# Training

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def cyclic_lr(epoch, lr):
    # Example function that modulates LR within a range for each epoch
    # Customize this function based on your cyclic learning rate policy
    max_lr = 0.01  # Maximum LR
    base_lr = 0.001  # Base LR
    step_size = 10  # Number of epochs for half a cycle
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * np.maximum(0, (1 - x))
    return lr

lr_scheduler = LearningRateScheduler(cyclic_lr)

In [11]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Convert activity targets to categorical
train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities)

# Train the model
history = model.fit(train_activity, 
                    train_targets_activity_cat,
                    epochs=25,
                    batch_size=64,
                    validation_data=(test_activity, test_targets_activity_cat),
                    verbose=1,
                    callbacks=[lr_scheduler])  # Add other callbacks as needed

Epoch 1/25
649/649 [==============================] - 5s 4ms/step - loss: 1.3487 - accuracy: 0.6855 - val_loss: 0.6314 - val_accuracy: 0.8257 - lr: 0.0010
Epoch 2/25
649/649 [==============================] - 2s 3ms/step - loss: 0.5375 - accuracy: 0.8464 - val_loss: 0.4794 - val_accuracy: 0.8537 - lr: 0.0019
Epoch 3/25
649/649 [==============================] - 2s 3ms/step - loss: 0.4646 - accuracy: 0.8565 - val_loss: 0.4415 - val_accuracy: 0.8604 - lr: 0.0028
Epoch 4/25
649/649 [==============================] - 2s 3ms/step - loss: 0.4386 - accuracy: 0.8613 - val_loss: 0.4269 - val_accuracy: 0.8647 - lr: 0.0037
Epoch 5/25
649/649 [==============================] - 2s 3ms/step - loss: 0.4240 - accuracy: 0.8634 - val_loss: 0.4132 - val_accuracy: 0.8655 - lr: 0.0046
Epoch 6/25
649/649 [==============================] - 2s 3ms/step - loss: 0.4157 - accuracy: 0.8651 - val_loss: 0.4069 - val_accuracy: 0.8675 - lr: 0.0055
Epoch 7/25
649/649 [==============================] - 2s 3ms/step - lo

In [12]:
# Evaluate the model using only the activity-related data
evaluation = model.evaluate(
    test_activity,
    test_targets_activity_cat
)

print(evaluation)

556/556 [==============================] - 1s 1ms/step - loss: 0.3946 - accuracy: 0.8668
[0.39464473724365234, 0.8668429255485535]


In [13]:
# Save the model to an H5 file
model.save('international.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

In [14]:
# Generate predictions for all inputs
predictions_activity = model.predict([windows_activity])

1852/1852 [==============================] - 2s 1ms/step


In [15]:
import numpy as np
from tensorflow.keras.utils import to_categorical

def compute_anomaly_scores(predictions, actuals):
    # For categorical predictions, convert actuals to one-hot for comparison
    actuals_one_hot = to_categorical(actuals, num_classes=predictions.shape[-1])
    
    max_predictions = np.max(predictions, axis=-1)
    actual_predictions = np.sum(predictions * actuals_one_hot, axis=-1)  # Extract the probability of the actual class
    
    anomaly_scores = (max_predictions - actual_predictions) / max_predictions
    
    return anomaly_scores

# Assuming targets_id, targets_resource, targets_activity, targets_role are the true values for these attributes
anomaly_scores_activity = compute_anomaly_scores(predictions_activity, targets_activity)

In [16]:
def classify_cases(anomaly_scores_activity, threshold=0.98):
    # Ensure all inputs are numpy arrays of the same shape
    anomaly_scores_activity = np.array(anomaly_scores_activity).flatten()

    # Find the maximum anomaly score across all attributes for each case
    max_scores = np.maximum.reduce([anomaly_scores_activity])

    # Classify cases as anomalous if the maximum anomaly score exceeds the threshold
    anomalous_cases = max_scores > threshold
    
    return anomalous_cases

# Now use the anomaly scores for id, resource, activity, role, and timestamp in the classification
anomalous_cases = classify_cases(anomaly_scores_activity)

# Mapping

### True: anomaly, False: no anomaly

In [17]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted'] = anomalous_cases
mapping

,case,predicted
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
59248,6448,False
59249,6448,False
59250,6448,False
59251,6448,False


In [18]:
case_prediction = mapping.groupby('case')['predicted'].any()
case_prediction


case
0       False
1       False
2       False
3       False
4       False
        ...  
6444    False
6445    False
6446    False
6447    False
6448    False
Name: predicted, Length: 6449, dtype: bool

# Ground Truth

In [19]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [20]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_InternationalDeclarations.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/753 [00:00<?, ?it/s]

In [21]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [22]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
6444,0,0
6445,1,0
6446,0,0
6447,1,0


# Evaluation

In [23]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [24]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.95


In [25]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.11


In [26]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.42


In [27]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.99


In [28]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.55
